In [1]:
import tushare as ts
from typing import Any

import numpy as np
import pandas as pd

pro = ts.pro_api('2026c96ef5fa7fc3241c96baafd638c585284c7fefaa00b93ef0a62c')

def code_to_secid(df : pd.DataFrame , code_col = 'ts_code' , retain = False):
    '''switch old symbol into secid'''
    if code_col not in df.columns.values: return df
    replace_dict = {'T00018' : '600018'}
    df['secid'] = df[code_col].astype(str).str.slice(0, 6).replace(replace_dict)
    df['secid'] = df['secid'].where(df['secid'].str.isdigit() , '-1').astype(int)
    if not retain: del df[code_col]
    return df

# adj0 = pro.query('adj_factor',  trade_date='20240705')


In [5]:
import numpy as np
import pandas as pd

from typing import Any

from src.data.download.tushare.download.abstract_fetcher import InfoFetcher
from src.data.download.tushare.basic import pro , code_to_secid

class FundInfo(InfoFetcher):
    DB_KEY = 'mutual_fund_info'
    UPDATE_FREQ = 'w'

    def get_data(self , date):
        renamer = {'ts_code' : 'fund_id'}
        df1 = self.iterate_fetch(pro.fund_basic , limit = 5000 , market='E')
        df2 = self.iterate_fetch(pro.fund_basic , limit = 5000 , market='O')

        df = pd.concat([df1 , df2]).rename(columns=renamer)
        for col in ['found_date' , 'due_date' , 'list_date' , 'issue_date' , 'delist_date']:
            df[col] = df[col].fillna(99991231).astype(int)
        df = df.reset_index(drop=True)
        return df

    
FundInfo().update()


d:\Coding\learndl\learndl\src\data\download\tushare\download\abstract_fetcher.py:96: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_df = pd.concat(dfs)
d:\Coding\learndl\learndl\src\data\download\tushare\download\abstract_fetcher.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(dfs).reset_index(drop = True)


DataBase object [information_ts],[mutual_fund_info],[20241211] save to D:\Coding\learndl\learndl\data\DataBase\DB_information_ts\mutual_fund_info.feather successfully


In [2]:
import numpy as np
import pandas as pd
from typing import Any

from src.basic import CALENDAR
from src.data.download.tushare.download.abstract_fetcher import InfoFetcher , TushareFetcher , updatable , quarter_ends
from src.data.download.tushare.basic import pro , code_to_secid

class FundPortfolioFetcher(TushareFetcher):
    START_DATE = 20070101
    DB_TYPE = 'fundport'
    UPDATE_FREQ = 'm'
    DB_SRC = 'holding_ts'
    DB_KEY = 'mutual_fund'
    DATA_FREQ = 'q'
    CONSIDER_FUTURE = False

    def update_dates(self):
        this_date , last_date , last_update_date = CALENDAR.today() , self.last_date() , self.last_update_date()

        update = updatable(this_date , last_update_date , self.UPDATE_FREQ)
        dates = quarter_ends(this_date , last_date , trailing_quarters = 1)  
        if not update and len(dates) <= 1: dates = []
        return dates
    
    def get_data(self , date):
        renamer = {'ts_code' : 'fund_id'}
        df = self.iterate_fetch(pro.fund_portfolio , limit = 3000 , period = str(date) , max_fetch_times=500)
        df = code_to_secid(df.rename(columns=renamer) , code_col='symbol' , retain = True)
        for col in ['ann_date' , 'end_date']:
            df[col] = df[col].fillna(99991231).astype(int)
        df = df.reset_index(drop=True)
        return df


#df = pro.fund_portfolio(period='20240930' , limit = 3000 , offset = 3000)
FundPortfolioFetcher().update()

src.basic.INSTANCE_RECORD can be accessed to check ['trainer', 'account']
Basic module imported!
DataBase object [holding_ts],[mutual_fund],[20210630] save to D:\Coding\learndl\learndl\data\DataBase\DB_holding_ts\mutual_fund\2021\mutual_fund.20210630.feather successfully
DataBase object [holding_ts],[mutual_fund],[20210930] save to D:\Coding\learndl\learndl\data\DataBase\DB_holding_ts\mutual_fund\2021\mutual_fund.20210930.feather successfully
DataBase object [holding_ts],[mutual_fund],[20211231] save to D:\Coding\learndl\learndl\data\DataBase\DB_holding_ts\mutual_fund\2021\mutual_fund.20211231.feather successfully
DataBase object [holding_ts],[mutual_fund],[20220331] save to D:\Coding\learndl\learndl\data\DataBase\DB_holding_ts\mutual_fund\2022\mutual_fund.20220331.feather successfully
DataBase object [holding_ts],[mutual_fund],[20220630] save to D:\Coding\learndl\learndl\data\DataBase\DB_holding_ts\mutual_fund\2022\mutual_fund.20220630.feather successfully
DataBase object [holding_ts]

In [7]:
FundPortfolioFetcher().update_dates()

array([20210630, 20210930, 20211231, 20220331, 20220630, 20220930,
       20221231, 20230331, 20230630, 20230930, 20231231, 20240331,
       20240630, 20240930])

In [2]:
df

,fund_id,ann_date,end_date,symbol,mkv,amount,stk_mkv_ratio,stk_float_ratio,secid
0,520700.SH,20241009,20240930,0013.HK,450083.39,15500.0,3.37,NaN,-1
1,520700.SH,20241009,20240930,1177.HK,978829.92,291000.0,7.32,NaN,-1
2,520700.SH,20241009,20240930,9926.HK,1113530.29,18000.0,8.33,NaN,-1
3,520700.SH,20241009,20240930,1093.HK,905667.70,166000.0,6.77,NaN,-1
4,520700.SH,20241009,20240930,1548.HK,477696.20,38000.0,3.57,NaN,-1
...,...,...,...,...,...,...,...,...,...
132037,002861.OF,20241025,20240930,002384.SZ,3152006.00,133900.0,5.78,0.01,2384
132038,002861.OF,20241025,20240930,002475.SZ,5228238.00,120300.0,9.59,0.00,2475
132039,002861.OF,20241025,20240930,300308.SZ,2115387.60,13660.0,3.88,0.00,300308
132040,517080.SH,20241025,20240930,2007.HK,NaN,203000.0,NaN,NaN,-1


In [5]:
tail([1,2,3])

NameError: name 'tail' is not defined

In [16]:
df[df['update_flag'] == 1]

,ts_code,ann_date,nav_date,unit_nav,accum_nav,accum_div,net_asset,total_netasset,adj_nav,update_flag


In [13]:
df = pro.fund_nav(ts_code='000001.OF')
df

,ts_code,ann_date,nav_date,unit_nav,accum_nav,accum_div,net_asset,total_netasset,adj_nav,update_flag
0,000001.OF,20241211,20241210,0.843,3.406,None,NaN,NaN,5.455472,0
1,000001.OF,20241210,20241209,0.841,3.404,None,NaN,NaN,5.442529,0
2,000001.OF,20241207,20241206,0.846,3.409,None,NaN,NaN,5.474886,0
3,000001.OF,20241206,20241205,0.839,3.402,None,NaN,NaN,5.429586,0
4,000001.OF,20241205,20241204,0.837,3.400,None,NaN,NaN,5.416643,0
...,...,...,...,...,...,...,...,...,...,...
5602,000001.OF,20020105,20020104,1.000,1.000,None,NaN,NaN,1.000000,0
5603,000001.OF,20020101,20011231,1.000,1.000,None,NaN,NaN,1.000000,0
5604,000001.OF,20011229,20011228,1.000,1.000,None,NaN,NaN,1.000000,0
5605,000001.OF,20011222,20011221,1.000,1.000,None,NaN,NaN,1.000000,0


In [14]:
df.dropna(subset=['total_netasset'])

,ts_code,ann_date,nav_date,unit_nav,accum_nav,accum_div,net_asset,total_netasset,adj_nav,update_flag
47,000001.OF,20241025,20240930,0.841,3.4040,None,2.578704e+09,2.578704e+09,5.442529,1
112,000001.OF,20240718,20240630,0.762,3.3250,None,2.353944e+09,2.353944e+09,4.931280,1
172,000001.OF,20240419,20240331,0.740,3.3028,None,2.295705e+09,2.295705e+09,4.788907,0
173,000001.OF,20240419,20240331,0.740,3.3030,None,2.295705e+09,2.295705e+09,4.788907,1
233,000001.OF,20240119,20231231,0.781,3.3440,None,2.420725e+09,2.420725e+09,5.054239,1
...,...,...,...,...,...,...,...,...,...,...
5377,000001.OF,20030102,20021231,0.945,0.9720,None,3.933701e+09,3.933701e+09,0.969100,0
5438,000001.OF,20021025,20020930,1.020,1.0470,None,4.269985e+09,4.269985e+09,1.046000,0
5504,000001.OF,20020829,20020630,1.073,1.1000,None,3.738304e+09,3.738304e+09,1.100300,0
5565,000001.OF,20020418,20020331,1.023,1.0230,None,3.254342e+09,3.254342e+09,1.023000,0
